In [ ]:
import sys
sys.path.insert(0,'../scripts')
from pycap_for_PESTPP_MOU import prepare_MOU_files, postprocess_MOU, plot_pareto, prep_for_viz, create_viz_app
from pathlib import Path
from ipywidgets import interact, fixed
import yaml



# Read in one of the configuration files and postprocess it

In [ ]:
config_file = 'MOU_fish_cash_baseline.yml'

with open(Path('./configurations') / config_file,'r') as ifp:
        inpars = yaml.safe_load(ifp)
inpars

In [ ]:
run_path = Path(inpars['run_path'])
run_name = inpars['scenario_name']

In [ ]:
pareto_df = postprocess_MOU(run_name, run_path)

In [ ]:
interact(plot_pareto,  currgen=(0,50,1), pareto_df=fixed(pareto_df));

In [ ]:
pareto_df_final, dv_df =  prep_for_viz(pareto_df, 50, run_path, run_name, dollar_objective=True)

In [ ]:
app = create_viz_app(pareto_df_final, dv_df)

In [ ]:
app.run(port=4242)